# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f2d001d7880>
├── 'a' --> tensor([[ 1.8312, -0.5588, -0.8938],
│                   [-0.5969, -0.9782, -0.0883]])
└── 'x' --> <FastTreeValue 0x7f2d001d7b20>
    └── 'c' --> tensor([[ 0.7187,  0.4468,  0.0656, -1.9203],
                        [-0.8002,  0.4117,  0.2384, -1.7923],
                        [-1.0609, -0.1370, -0.3257,  1.2616]])

In [4]:
t.a

tensor([[ 1.8312, -0.5588, -0.8938],
        [-0.5969, -0.9782, -0.0883]])

In [5]:
%timeit t.a

63.4 ns ± 0.141 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f2d001d7880>
├── 'a' --> tensor([[ 1.8520, -1.9909,  0.2433],
│                   [ 0.8368,  1.3521, -1.5298]])
└── 'x' --> <FastTreeValue 0x7f2d001d7b20>
    └── 'c' --> tensor([[ 0.7187,  0.4468,  0.0656, -1.9203],
                        [-0.8002,  0.4117,  0.2384, -1.7923],
                        [-1.0609, -0.1370, -0.3257,  1.2616]])

In [7]:
%timeit t.a = new_value

80.1 ns ± 0.0606 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.8312, -0.5588, -0.8938],
               [-0.5969, -0.9782, -0.0883]]),
    x: Batch(
           c: tensor([[ 0.7187,  0.4468,  0.0656, -1.9203],
                      [-0.8002,  0.4117,  0.2384, -1.7923],
                      [-1.0609, -0.1370, -0.3257,  1.2616]]),
       ),
)

In [10]:
b.a

tensor([[ 1.8312, -0.5588, -0.8938],
        [-0.5969, -0.9782, -0.0883]])

In [11]:
%timeit b.a

52 ns ± 0.0632 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.6137,  0.6109, -0.3217],
               [ 1.8421, -2.0891, -1.7462]]),
    x: Batch(
           c: tensor([[ 0.7187,  0.4468,  0.0656, -1.9203],
                      [-0.8002,  0.4117,  0.2384, -1.7923],
                      [-1.0609, -0.1370, -0.3257,  1.2616]]),
       ),
)

In [13]:
%timeit b.a = new_value

492 ns ± 0.0339 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

859 ns ± 0.671 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11 µs ± 8.44 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

146 µs ± 1.95 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 237 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f2d001d3370>
├── 'a' --> tensor([[[ 1.8312, -0.5588, -0.8938],
│                    [-0.5969, -0.9782, -0.0883]],
│           
│                   [[ 1.8312, -0.5588, -0.8938],
│                    [-0.5969, -0.9782, -0.0883]],
│           
│                   [[ 1.8312, -0.5588, -0.8938],
│                    [-0.5969, -0.9782, -0.0883]],
│           
│                   [[ 1.8312, -0.5588, -0.8938],
│                    [-0.5969, -0.9782, -0.0883]],
│           
│                   [[ 1.8312, -0.5588, -0.8938],
│                    [-0.5969, -0.9782, -0.0883]],
│           
│                   [[ 1.8312, -0.5588, -0.8938],
│                    [-0.5969, -0.9782, -0.0883]],
│           
│                   [[ 1.8312, -0.5588, -0.8938],
│                    [-0.5969, -0.9782, -0.0883]],
│           
│                   [[ 1.8312, -0.5588, -0.8938],
│                    [-0.5969, -0.9782, -0.0883]]])
└── 'x' --> <FastTreeValue 0x7f2d001d3820>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 25.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f2c556023a0>
├── 'a' --> tensor([[ 1.8312, -0.5588, -0.8938],
│                   [-0.5969, -0.9782, -0.0883],
│                   [ 1.8312, -0.5588, -0.8938],
│                   [-0.5969, -0.9782, -0.0883],
│                   [ 1.8312, -0.5588, -0.8938],
│                   [-0.5969, -0.9782, -0.0883],
│                   [ 1.8312, -0.5588, -0.8938],
│                   [-0.5969, -0.9782, -0.0883],
│                   [ 1.8312, -0.5588, -0.8938],
│                   [-0.5969, -0.9782, -0.0883],
│                   [ 1.8312, -0.5588, -0.8938],
│                   [-0.5969, -0.9782, -0.0883],
│                   [ 1.8312, -0.5588, -0.8938],
│                   [-0.5969, -0.9782, -0.0883],
│                   [ 1.8312, -0.5588, -0.8938],
│                   [-0.5969, -0.9782, -0.0883]])
└── 'x' --> <FastTreeValue 0x7f2c54f0a580>
    └── 'c' --> tensor([[ 0.7187,  0.4468,  0.0656, -1.9203],
                        [-0.8002,  0.4117,  0.2384, -1.7923],
                 

In [23]:
%timeit t_cat(trees)

30.3 µs ± 66.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59 µs ± 31.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.7187,  0.4468,  0.0656, -1.9203],
                       [-0.8002,  0.4117,  0.2384, -1.7923],
                       [-1.0609, -0.1370, -0.3257,  1.2616]],
              
                      [[ 0.7187,  0.4468,  0.0656, -1.9203],
                       [-0.8002,  0.4117,  0.2384, -1.7923],
                       [-1.0609, -0.1370, -0.3257,  1.2616]],
              
                      [[ 0.7187,  0.4468,  0.0656, -1.9203],
                       [-0.8002,  0.4117,  0.2384, -1.7923],
                       [-1.0609, -0.1370, -0.3257,  1.2616]],
              
                      [[ 0.7187,  0.4468,  0.0656, -1.9203],
                       [-0.8002,  0.4117,  0.2384, -1.7923],
                       [-1.0609, -0.1370, -0.3257,  1.2616]],
              
                      [[ 0.7187,  0.4468,  0.0656, -1.9203],
                       [-0.8002,  0.4117,  0.2384, -1.7923],
                       [-1.0609, -0.1370, -0.3257,  1.2616]],

In [26]:
%timeit Batch.stack(batches)

77.1 µs ± 202 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.7187,  0.4468,  0.0656, -1.9203],
                      [-0.8002,  0.4117,  0.2384, -1.7923],
                      [-1.0609, -0.1370, -0.3257,  1.2616],
                      [ 0.7187,  0.4468,  0.0656, -1.9203],
                      [-0.8002,  0.4117,  0.2384, -1.7923],
                      [-1.0609, -0.1370, -0.3257,  1.2616],
                      [ 0.7187,  0.4468,  0.0656, -1.9203],
                      [-0.8002,  0.4117,  0.2384, -1.7923],
                      [-1.0609, -0.1370, -0.3257,  1.2616],
                      [ 0.7187,  0.4468,  0.0656, -1.9203],
                      [-0.8002,  0.4117,  0.2384, -1.7923],
                      [-1.0609, -0.1370, -0.3257,  1.2616],
                      [ 0.7187,  0.4468,  0.0656, -1.9203],
                      [-0.8002,  0.4117,  0.2384, -1.7923],
                      [-1.0609, -0.1370, -0.3257,  1.2616],
                      [ 0.7187,  0.4468,  0.0656, -1.9203],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 1.42 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

327 µs ± 1.06 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
